In [2]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification
import spacy

In [3]:
import warnings
warnings.filterwarnings('ignore')

In [4]:
# Load the model
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=3)
checkpoint = torch.load('f_arith_clf.pth')
model.load_state_dict(checkpoint['model_state_dict'])
model.eval()

# Load the tokenizer
tokenizer = BertTokenizer.from_pretrained('bert_fuzzy_arithmetic_tokenizer')

A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Pl

In [5]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [6]:
def predict_arithmetic_type(text, model, tokenizer, device):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
    inputs = {k: v.to(device) for k, v in inputs.items()}
    model.eval()
    
    # Make prediction
    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits
        predicted_class = torch.argmax(logits, dim=1).item()
    
    # Mapping the predicted class to the arithmetic type
    class_names = ['addition', 'subtraction', 'multiplication']
    return class_names[predicted_class]

In [7]:
import spacy
import numerizer
from spacy.language import Language
from spacy.tokens import Doc, Span, SpanGroup
import pint

ureg = pint.UnitRegistry()
ureg.define('USD = [currency] = dollar')
ureg.define('cent = 0.01 * USD')

# Register the language components
@Language.component("numerizer_component")
def numerizer_component(doc):
    numerized_text = numerizer.numerize(doc.text)
    return Doc(doc.vocab, words=numerized_text.split())

@Language.component("fuzzy_entities")
def fuzzy_entities(doc):
    span_group = doc.spans["fuzzy_ents"] = SpanGroup(doc)
    fuzzy_term_pos = ["ADV", "ADJ", "NOUN", "ADP"]
    fuzzy_range_pos = ["CCONJ"]
    fuzzy_range_terms = ["to"] # ADP that can be used as a range term
    for i in range(len(doc) - 1):
        if i> 0 and (doc[i].like_num or doc[i].text.isdigit()): 
            if doc[i - 1].pos_ in fuzzy_term_pos:
                span_group.append(Span(doc, i - 1, i, label="FUZZY_TERM"))
                if i < (len(doc)-1) \
                        and doc[i + 1].pos_ not in fuzzy_range_pos \
                        and doc[i + 1].text not in fuzzy_range_terms:
                    span_group.append(Span(doc, i, i + 1, label="FUZZY_VALUE"))
            if (doc[i - 1].pos_ in fuzzy_range_pos
                or doc[i - 1].text in fuzzy_range_terms) \
                    and (doc[i - 2].like_num or doc[i - 2].text.isdigit()):
                span_group.append(Span(doc, i - 2, i + 1, label="FUZZY_RANGE"))
        try:
            ureg(doc[i].text)
            if i > 0 and (doc[i - 1].like_num or doc[i - 1].text.isdigit()):
                span_group.append(Span(doc, i, i + 1, label="MEASUREMENT_UNIT"))
        except:
            continue

    return doc

In [11]:
# Load the pipeline
nlp = spacy.load("f_values.pth")

In [12]:
text = "If I have around 5 apples and I get at least 3 more, how many apples do I have in total?"

In [13]:
prediction = predict_arithmetic_type(text, model, tokenizer, device)
print(f"The predicted arithmetic type is: {prediction}")

The predicted arithmetic type is: addition


In [14]:
doc = nlp(text)
[(ent.text, ent.label_) for ent in doc.spans["fuzzy_ents"]]

[('around', 'FUZZY_TERM'),
 ('5', 'FUZZY_VALUE'),
 ('least', 'FUZZY_TERM'),
 ('3', 'FUZZY_VALUE')]